In [279]:
import cv2 as cv
from enum import Enum
import numpy as np
import builtins

computeErrorSurface:
    img1 is always on top or on left of img2.

In [280]:
class Direction(Enum):
    VERTICAL = 0
    HORIZONTAL = 1

In [281]:
def computeErrorSurface(img1, img2, boundarySize: int, direction: Direction):
    border1 = border2 = None
    if direction == Direction.HORIZONTAL:
        border1 = img1[-boundarySize-1:-1,:]
        border2 = img2[0:boundarySize,:]
    else:
        border1 = img1[:,-boundarySize-1:-1]
        border2 = img2[:,0:boundarySize]

    diff = np.subtract(border1, border2)

    squareDiff = np.square(diff)

    ret = np.zeros((squareDiff.shape[0], squareDiff.shape[1]),dtype=np.uint32)

    for i in range(ret.shape[0]):
        for j in range(ret.shape[1]):
            ret[i,j] = np.sum(squareDiff[i,j])

    ret = np.sqrt(ret)
    return cv.normalize(ret,None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)


In [282]:
def verticalNeighbors(currentPos, errorBoundary):

    neighbors = []

    if(currentPos[0] != errorBoundary.shape[0]):
        neighbors.append((currentPos[0] - 1, currentPos[1]))
        if(currentPos[1] != 0):
            neighbors.append((currentPos[0] - 1, currentPos[1] - 1))
        if(currentPos[1] != errorBoundary.shape[1] - 1):
            neighbors.append((currentPos[0] - 1, currentPos[1] + 1))
    else:
        for i in range(errorBoundary.shape[1]):
            neighbors.append((currentPos[0] - 1, i))

    return neighbors

def horizontalNeighbors(currentPos, errorBoundary):

    neighbors = []

    if(currentPos[1] != errorBoundary.shape[1]):
        neighbors.append((currentPos[0], currentPos[1] - 1))
        if(currentPos[0] != 0):
            neighbors.append((currentPos[0] - 1, currentPos[1] - 1))
        if(currentPos[0] != errorBoundary.shape[0] - 1):
            neighbors.append((currentPos[0] + 1, currentPos[1] - 1))
    else:
        for i in range(errorBoundary.shape[0]):
            neighbors.append((i, currentPos[1] - 1))

    return neighbors

In [283]:
# function heavily inspired by Ben Alex Keen's implementation: https://benalexkeen.com/implementing-djikstras-shortest-path-algorithm-with-python/
def minimumBoundaryCutDjikstras(errorBoundary, direction: Direction):
    dirIndex = direction.value
    invDirIndex = (direction.value + 1) % 2
    initial = [0,0]
    initial[dirIndex] = errorBoundary.shape[dirIndex]
    initial = tuple(initial)
    
    neighborFunction = verticalNeighbors

    if(direction == Direction.HORIZONTAL):
        neighborFunction = horizontalNeighbors

    shortestPaths = {initial: (None, 0)}
    currentNode = initial
    visited = set()
    
    while currentNode[dirIndex] != 0:
        visited.add(currentNode)
        destinations = neighborFunction(currentNode, errorBoundary)
        weightToCurrentNode = shortestPaths[currentNode][1]

        for nextNode in destinations:
            weight = errorBoundary[nextNode[0], nextNode[1]] + weightToCurrentNode
            if nextNode not in shortestPaths:
                shortestPaths[nextNode] = (currentNode, weight)
            else:
                currentShortestWeight = shortestPaths[nextNode][1]
                if currentShortestWeight > weight:
                    shortestPaths[nextNode] = (currentNode, weight)
        
        nextDestinations = {node: shortestPaths[node] for node in shortestPaths if node not in visited}
        currentNode = builtins.min(nextDestinations, key=lambda k: nextDestinations[k][1])
    
    path = []
    while currentNode is not None:
        path.append(currentNode[invDirIndex])
        nextNode = shortestPaths[currentNode][0]
        currentNode = nextNode

    return path[:-1]

In [284]:
def show(names, imgs, scaling = 1):
    for i in range(len(names)):
        cv.imshow(names[i], cv.resize(imgs[i], (imgs[i].shape[1] * scaling, imgs[i].shape[0] * scaling)))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [285]:
def applyWithMask(fullImg, patch, position, boundarySize, cutUp = None, cutLeft = None):
    size = patch.shape[0] - boundarySize
    pixelPosition = [position[0]*size, position[1]*size]
    
    patchForPasting = cv.cvtColor(patch, cv.COLOR_RGB2RGBA)
    
    patchForPasting[:,:,3] = 255
    
    if(cutUp != None):
        for i in range(patch.shape[0]):
            patchForPasting[:cutUp[i],i,3] = 0

    if(cutLeft != None):
        for i in range(patch.shape[0]):
            patchForPasting[i,:cutLeft[i],3] = 0
  
    fullImg[pixelPosition[0]:pixelPosition[0] + patch.shape[0], pixelPosition[1]:pixelPosition[1] + patch.shape[1]]  = fullImg[pixelPosition[0]:pixelPosition[0] + patch.shape[0], pixelPosition[1]:pixelPosition[1] + patch.shape[1]] * (1 - patchForPasting[:, :, 3:] / 255) + patchForPasting[:, :, :3]  * (patchForPasting[:, :, 3:] / 255)
    return fullImg

In [286]:
def computeMinimumBoundaryCut(currentImg, boundarySize, upImg = None, leftImg = None):
    errorBoundaries = [None, None]

    if(upImg is not None):
        e = computeErrorSurface(upImg, currentImg, boundarySize, Direction.HORIZONTAL)
        errorBoundaries[Direction.HORIZONTAL.value] = e

    if(leftImg is not None):
        e = computeErrorSurface(leftImg, currentImg, boundarySize, Direction.VERTICAL)
        errorBoundaries[Direction.VERTICAL.value] = e

    if(errorBoundaries[0] is not None and errorBoundaries[1] is not None):
        minima = np.minimum(errorBoundaries[0][:boundarySize, :boundarySize], errorBoundaries[1][:boundarySize, :boundarySize])
        errorBoundaries[0][:boundarySize, :boundarySize] = errorBoundaries[1][:boundarySize, :boundarySize] = minima

    if(upImg is not None):
        e = errorBoundaries[Direction.HORIZONTAL.value]
        minCut = minimumBoundaryCutDjikstras(e, Direction.HORIZONTAL)
        errorBoundaries[Direction.HORIZONTAL.value] = (minCut, e)

    if(leftImg is not None):
        e = errorBoundaries[Direction.VERTICAL.value]
        minCut = minimumBoundaryCutDjikstras(e, Direction.VERTICAL)
        errorBoundaries[Direction.VERTICAL.value] = (minCut, e)

    return errorBoundaries

In [ ]:
def getCutImage(errorSurface, cut, index):
    cutImage = cv.merge((errorSurface,errorSurface,errorSurface))
    for i in range(cutImage.shape[index]):
        cutImage[i, cut[i]] = [0,255,0]
    return cutImage

In [287]:
img2 = cv.imread("img1.jpg", cv.IMREAD_COLOR)
img1 = cv.imread("img2.png", cv.IMREAD_COLOR)

patches = [img1, img2]

grid = [[1, 2, 1],
        [2, 1, 2],
        [1, 2, 1]]

#fazer dicionário -> curId, upId, leftId -> upCut, leftCut, upId ou leftId podem ser None caso não existam
#ver uma celula, passar ids e ver se dicionario existe
#se não existe, calcular cortes e salvar no dicionario

#construir imagem completa, recuperando cortes do dicionário a cada célula


borderSize = 50

for 

minBCut, _  = computeMinimumBoundaryCut(img2, borderSize, leftImg=img1)[Direction.VERTICAL.value]
minBCut2, _ = computeMinimumBoundaryCut(img2, borderSize, upImg=img1)[Direction.HORIZONTAL.value]
temp = computeMinimumBoundaryCut(img1, borderSize, leftImg=img2, upImg=img2)
minBCut3 = temp[Direction.VERTICAL.value][0]
minBCut4 = temp[Direction.HORIZONTAL.value][0]

cellSize = img1.shape[0] - borderSize

canvas = np.zeros((cellSize*2 + borderSize, cellSize*2 + borderSize, 3), dtype=np.uint8)

canvas = applyWithMask(canvas, img1, (0,0), borderSize)
canvas = applyWithMask(canvas, img2, (0,1), borderSize, cutLeft=minBCut)
canvas = applyWithMask(canvas, img2, (1,0), borderSize, cutUp=minBCut2)
canvas = applyWithMask(canvas, img1, (1,1), borderSize, cutLeft = minBCut3, cutUp=minBCut4)

show(["image 1", "image 2", "boundary error up", "boundary cut up","boundary error dpwn", "boundary cut down", "canvas"], [img1, img2, e1, minBCut3d, e2, minBCut4d, canvas], 2)